## Generate a PSD plot framework for future plots.

In [6]:
#Purisa Jasmine Simmons
#Viren Abhyankar
#February 2020

#Overview: Trying to generate a PSD plot (framework for future plots).
#Based on Method V of this paper: https://journals.ametsoc.org/doi/pdf/10.1175/2010JTECHO724.1

#First, parse the data from the .CSV file.
#This data comes from Scripps buoy that recalibrates every 30 minutes 
#all of the vertical accelerations are contained in IMUA2.

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from mpl_toolkits.basemap import Basemap

import pandas as pd
import numpy as np

from scipy import stats
from scipy import constants
from scipy import signal #added
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d
from scipy.integrate import simps
from scipy.integrate import cumtrapz

import os
import datetime
import pytz
import re

import peakutils
import statsmodels.api as sm

import requests


from plotly import tools #added all the plotly's
import plotly.offline
import plotly.graph_objs as go

import math #added
import re   #added

# For the definition of the abstract base class IMU_Base
import abc

import sys

# %matplotlib notebook
%matplotlib inline


In [11]:
## Read from CSV
#Full csv:
#scripps_final_combined = pd.read_csv('Scripps_Final_Combined.csv')
#print(scripps_final_combined.head())

#Gets all columns
motion_df = pd.read_csv('Scripps_Final_Combined.csv')
#motion_df = motion_df.drop(['Time'], axis=1)
print(motion_df.head())
print(len(motion_df))

   Unnamed: 0                                                 ID  \
0           0  google_106807627751629413403_0006667E228C_1709...   
1           1  google_106807627751629413403_0006667E228C_1709...   
2           2  google_106807627751629413403_0006667E228C_1709...   
3           3  google_106807627751629413403_0006667E228C_1709...   
4           4  google_106807627751629413403_0006667E228C_1709...   

                              UTC   Time    IMU A1     IMU A2    IMU A3  \
0  2017-09-09T18:02:11.2810+00:00  18:02  0.000000  11.070788  0.134075   
1  2017-09-09T18:02:11.4800+00:00  18:02 -0.325611  11.185710 -0.459687   
2  2017-09-09T18:02:11.7310+00:00  18:02  0.000000  10.419566  1.800440   
3  2017-09-09T18:02:11.9810+00:00  18:02  0.823605  10.860099 -0.114922   
4  2017-09-09T18:02:12.1800+00:00  18:02  0.670376  11.492168 -0.651223   

      IMU G1     IMU G2     IMU G3  IMU M1  IMU M2  IMU M3        Date  \
0  46.829268  15.853659  15.487805   -37.0  -199.0   517.0  2017-0

## Make a copy of the original motion_df dataframe

In [8]:
#print(motion_df)

saved_copy_motion_df = motion_df.copy(deep=True) #make a copy of the dataframe with raw data included

#print(saved_copy_motion_df)

## Calcualting time_elapseds, time_offsets and creating IMU1, IMU2, and IMU3 raw data lists

In [33]:
#Reading data from Scripps_Final_Combined.csv
#The name of the motion dataframe is: motion_df

#Get times from the "Time" column to create time_o_list and time_e_list.
#Get imus from the "IMU A[*]" column to create the imu acc arrays. 

#Analyze following time period:
#Can use np.where()[0][0] for the following:
#9/9/2017 - 18:10 to 18:40
#6/14/2018 - 15:20 to 16:20

#Analyze particular UTC with actual heights (m) -- 30 minute sessions
# 2017-09-09T18:24:00.1810+00:00 to 2017-09-09T18:54:00.1810+00:00 -- 0.810768
# 2017-09-09T18:54:00.1810+00:00 to 2018-06-14T15:13:06.7140+00:00 -- 0.819912
# 2018-06-14T15:13:06.7140+00:00 to 2018-06-14T15:43:00.0050+00:00 -- 0.679704
# 2018-06-14T15:43:00.0050+00:00 to 2018-06-14T16:13:00.2150+00:00 -- 0.70104
# 2019-05-29T15:21:41.0170+00:00 to 2019-05-29T15:51:00.0140+00:00 -- 1.069848
# 2019-05-29T15:51:00.0140+00:00 to 2019-05-29T16:21:00.0620+00:00 -- 1.179576
# 2019-05-29T16:21:00.0620+00:00 to 2019-05-29T16:51:00.1630+00:00 -- 1.179576

START = '2019-05-29T16:21:00.0620+00:00'
END = '2019-05-29T16:51:00.1630+00:00'

time_e_array = []

time_array = motion_df[['Time']].to_numpy()
utc_time = motion_df[['UTC']].to_numpy()

start_index = np.where(utc_time==START)[0][0]
end_index = np.where(utc_time==END)[0][0]

#Convert time strings into seconds
#start_index = np.where(time_array=='15:20')[0][0]
#end_index = np.where(time_array=='16:20')[0][0]

length = end_index-start_index
print(length)

mins_per_measure = 60/length
sec_per_measure = 60*mins_per_measure

print(sec_per_measure)

#start_time = 18*3600 + 10*60

for i in range(0, length):
    time_e_array.append(sec_per_measure*i)
    
imu_array1 = motion_df.loc[start_index:end_index-1,'IMU A1'].to_numpy()
imu_array2 = motion_df.loc[start_index:end_index-1,'IMU A2'].to_numpy()
imu_array3 = motion_df.loc[start_index:end_index-1,'IMU A3'].to_numpy()

55182
7551
0.4767580452920143


# Parse through dataset

In [94]:
# Function to parse time string and calculate half hour
def add_half_hour(time_str):
    hrs = int(time_str[:2])
    mins = int(time_str[3:5])
    if (mins < 30):
        mins += 30
    else:
        mins -= 30
        if (hrs == 23):
            hrs = 0
        else:
            hrs += 1
            
    if (mins < 10):
        mins = '0'+str(mins)
    if (hrs < 10):
        hrs = '0'+str(hrs)
        
    return str(hrs)+':'+str(mins)

print(add_half_hour('18:53'))

19:23


In [127]:
### Goal: create an array of arrays of IMU A2 data ###
surf_sessions = []

## Step 1: get 5 columns of the data frame to iterate over - ID, Time, IMU A2, Date, Height ##
df_parser = motion_df[['ID', 'Time', 'IMU A2', 'Date', 'Hs(m)']]

id_parser = df_parser[['ID']].to_numpy()
time_parser = df_parser[['Time']].to_numpy()
imuA2_parser = df_parser[['IMU A2']].to_numpy()
date_parser = df_parser[['Date']].to_numpy()
height_parser = df_parser[['Hs(m)']].to_numpy()

## Step 2: loop through every data point ##
working_array = []
skip_check = '00:00'
end_index_check = None

for start_index in range(0, len(id_parser)):
    ## Goal 1: Append IMU data if we are in the half-hour segment ##
    
    if (end_index_check != None):
        # Check if we've reached the end of the half hour before appending
        if (start_index==end_index_check):
            surf_sessions.append(working_array)
            working_array = []
            end_index_check = None
        else:
            working_array.append(imuA2_parser[start_index][0])
        continue
        
    ## Goal 2: Check if height of current and half-hour times are equal ##
    
    start = time_parser[start_index][0]
    
    # Skip through a minute if it has already been checked and invalidated
    if (start==skip_check):
        continue
        
    end = add_half_hour(start)
    
    # Check if ride is still valid for half hour time
    try:
        end_index = np.where((time_parser==end) & (id_parser==id_parser[start_index]))[0][0]
        # Reset if ride is valid
    except IndexError:
        skip_check = start
        continue
    
    if (height_parser[start_index]!=height_parser[end_index-1]):
        skip_check = start
    else:
        working_array.append(imuA2_parser[start_index][0])
        end_index_check = end_index
        
        print("Session: ", len(surf_sessions)+1, " and Ride: ", date_parser[start_index])
        print("At ", start, " and ", end)
        print("Start: ", height_parser[start_index], " and End: ", height_parser[end_index-1], "\n")
    
print("Total sessions: ", len(surf_sessions))


Session:  1  and Ride:  ['2017-09-09']
At  18:10  and  18:40
Start:  [0.810768]  and End:  [0.810768] 

Session:  2  and Ride:  ['2017-09-09']
At  18:40  and  19:10
Start:  [0.819912]  and End:  [0.819912] 

Session:  3  and Ride:  ['2018-06-14']
At  15:20  and  15:50
Start:  [0.70104]  and End:  [0.70104] 

Session:  4  and Ride:  ['2018-06-14']
At  15:50  and  16:20
Start:  [0.710184]  and End:  [0.710184] 

Session:  5  and Ride:  ['2019-05-29']
At  15:50  and  16:20
Start:  [1.069848]  and End:  [1.069848] 

Session:  6  and Ride:  ['2019-05-29']
At  16:20  and  16:50
Start:  [1.179576]  and End:  [1.179576] 

Session:  7  and Ride:  ['2018-08-12']
At  14:46  and  15:16
Start:  [0.618744]  and End:  [0.618744] 

Session:  8  and Ride:  ['2018-08-12']
At  15:20  and  15:50
Start:  [0.649224]  and End:  [0.649224] 

Session:  9  and Ride:  ['2017-08-08']
At  16:40  and  17:10
Start:  [0.920496]  and End:  [0.920496] 

Session:  10  and Ride:  ['2017-08-08']
At  17:10  and  17:40
Star

Session:  81  and Ride:  ['2018-09-10']
At  17:50  and  18:20
Start:  [0.89916]  and End:  [0.89916] 

Session:  82  and Ride:  ['2018-09-10']
At  18:20  and  18:50
Start:  [0.938784]  and End:  [0.938784] 

Session:  83  and Ride:  ['2017-07-02']
At  02:10  and  02:40
Start:  [0.688848]  and End:  [0.688848] 

Session:  84  and Ride:  ['2017-07-02']
At  02:40  and  03:10
Start:  [0.70104]  and End:  [0.70104] 

Session:  85  and Ride:  ['2017-12-04']
At  19:05  and  19:35
Start:  [0.86868]  and End:  [0.86868] 

Session:  86  and Ride:  ['2017-12-04']
At  19:40  and  20:10
Start:  [0.920496]  and End:  [0.920496] 

Session:  87  and Ride:  ['2018-11-04']
At  21:20  and  21:50
Start:  [0.6096]  and End:  [0.6096] 

Session:  88  and Ride:  ['2018-11-04']
At  21:50  and  22:20
Start:  [0.560832]  and End:  [0.560832] 

Session:  89  and Ride:  ['2019-02-08']
At  17:50  and  18:20
Start:  [0.469392]  and End:  [0.469392] 

Session:  90  and Ride:  ['2019-02-08']
At  18:20  and  18:50
Sta

## Convert raw units to actual units (acc to [m/s^2]) and (time to [s])

In [ ]:
## Convert raw units to actual units (acc to [m/s^2]) and (time to [s])


#Raw acceleration constant 512 = 1g (accelerometer's measured force due to gravity)
g_const = 512

#Approximate measurement for gravity:
gravity = 9.80665


# Correct the IMU Acceleration columns into units of meters
# Dividing by 512 is equivalent to muliplying by 4 to correct the bit shifting by 2 places and dividing by 2048 to convert bits to G's
# Multiplying by the 9.81 afterwards is simply to convert G's into m/s^2

def convert_acc_units(acc_array):
    ret_array = []
    for a in acc_array:
        #Acceleration is now in m/s^2, need to subtract gravity from vertical axis. (??)
        new_a = a / g_const * gravity + gravity
        ret_array.append(new_a)
    return ret_array

#imu1_array = convert_acc_units(imu_array1) #new units in m/s^2
#imu2_array = convert_acc_units(imu_array2) #new units in m/s^2
#imu3_array = convert_acc_units(imu_array3) #new units in m/s^2
imu1_array = imu_array1
imu2_array = imu_array2
imu3_array = imu_array3

##To check:
#for x,y in zip(imu2_array, imu_array2):
#    print(x,y)

##To check:
#for t in time_e_array:
#    print(t)


print("Graph of our entire experiment:")

plt.plot(time_e_array, imu2_array)
#plt.ylim(-1,1)
plt.show()

#print("Why are the y-axis values so small?")

## Step 1: Seperate each of the subexperiments into its own acc lists

In [ ]:
#for x in time_e_array:
#    print(x)

In [ ]:
#Seperate each of the subexperiments into its own acc lists.
#i.e. subexperiment1 corresponds to acc1, (subexperiment2 => acc2), etc.

time_e_list1 = []
time_e_list2 = []
time_e_list3 = []

acc_list = []
acc_list1 = []
acc_list2 = []
acc_list3 = []

time_array = []
acc_array = []


#For our controlled experiments, we know that imu2 is the vertical axis
acc_list = imu2_array
time_array = time_e_array


#i = 0
#while (i < len(imu2_array)):
#    imu2_array[i] = imu2_array[i] - gravity
#    i = i + 1
#print(imu2_array)

i = 0
while (i < (len(acc_list)) - 1):
    if (time_e_array[i] >= 0 and time_e_array[i] <= 1200):
        acc_list1.append(acc_list[i])
        time_e_list1.append(time_e_array[i])
    if (time_e_array[i] > 1200 and time_e_array[i] <= 2400):
        acc_list2.append(acc_list[i])
        time_e_list2.append(time_e_array[i])
    if (time_e_array[i] > 2400 and time_e_array[i] <= 3600):
        acc_list3.append(acc_list[i])
        time_e_list3.append(time_e_array[i])
    i = i + 1
    
#Plot the subexperiments to verify correctness:
acc_array = np.array(acc_list)   #acc_list gets vertical acceleration from y-axis imu2


time_array1 = np.array(time_e_list1)
acc_array1 = np.array(acc_list1)
time_array2 = np.array(time_e_list2)
acc_array2 = np.array(acc_list2)
time_array3 = np.array(time_e_list3)
acc_array3 = np.array(acc_list3)
  
    
##Plotting:
f1 = plt.figure(figsize=(10,3))
ax1 = f1.add_subplot(121)
ax2 = f1.add_subplot(122)

ax1.plot(time_array, acc_array)
ax1.set_title("Acceleration vs. Time")
ax1.set_xlabel("Time [s]")
ax1.set_ylabel('Acceleration [m/s^2]')


ax2.plot(time_array1, acc_array1)
ax2.set_title("Acceleration1 vs. Time")
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Acceleration [m/s^2]")
#ax2.set_ylim([-6,6])


plt.show()
    

## Step 2: Detrend and Double Integrate the data 

In [ ]:
##PSD Step 2: Detrend the data 
dacc_array1 = signal.detrend(acc_array1)
dacc_array2 = signal.detrend(acc_array2)
dacc_array3 = signal.detrend(acc_array3)

#f_s = 5.0 #sampling frequency


### Bandpass filter the data

In [ ]:
#First integral of acc to get velocity:
from scipy import integrate

def calculate_new_range(time_array, array_of_values, low_time, high_time):
    new_time_array = []
    new_value_array = []
    for t,v in zip(time_array, array_of_values):
        if (t >= low_time and t < high_time):
            new_time_array.append(t)
            new_value_array.append(v)
    return new_time_array, new_value_array


In [ ]:
## Butter Filters for Bandpass:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_lfilter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

def butter_bandpass_filtfilt(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y



##Butter Filters for Highpass:
def butter_highpass(highcut, fs, order=5):
    nyq = 0.5 * fs
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='high')
    return b, a

def butter_highpass_lfilter(data, highcut, fs, order=5):
    b, a = butter_lowpass(highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y



##Butter Filters for Lowpass:
def butter_lowpass(lowcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    b, a = signal.butter(order, low, btype='low')
    return b, a

def butter_lowpass_lfilter(data, lowcut, fs, order=5):
    b, a = butter_lowpass(lowcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y




# Define the sample rate and the Low and High Cutoff frequencies
fs = 30
lowcut = 0.0333
highcut = 1.5

##Graphing the bandpass filters:
butter_lfilter = butter_bandpass_lfilter(dacc_array1, lowcut, highcut, fs, order=5)
butter_filtfilt = butter_bandpass_filtfilt(dacc_array1, lowcut, highcut, fs, order=5)


###     Following code is NEVER USED 
'''
new_time = time_array1.copy()

for i in range(len(new_time)):
    if i == 0:
        new_time[i] = 0
    else:
        new_time[i] = new_time[i-1] + 1/fs
'''
### 
        


fig1 = plt.figure(figsize=(15,8))
ax1 = fig1.add_subplot(131)
ax2 = fig1.add_subplot(132)
ax3 = fig1.add_subplot(133)

ax1.plot(time_array1, dacc_array1)
ax1.set_title('Oritinal detrended Acceleration1')
ax1.set_xlabel('Relative Time [sec]')
ax1.set_ylabel('Amplitude')
ax1.axhline(0, color="orange",ls='--')


ax2.plot(time_array1, butter_lfilter)
ax2.set_title('Butterpass Z Acceleration1 LFilter')
ax2.set_xlabel('Relative Time [sec]')
ax2.set_ylabel('Amplitude')
ax2.axhline(0, color="orange",ls='--')

ax3.plot(time_array1, butter_filtfilt)
ax3.set_title('Butterpass Acceleration1 Filfilt')
ax3.set_xlabel('Relative Time [sec]')
ax3.set_ylabel('Amplitude')
ax3.axhline(0, color="orange",ls='--')


plt.show()

In [ ]:
# Define the sample rate and the Low and High Cutoff frequencies

fs = 5 #redefine the sampling frequency

order=6
lowcut = 0.09
highcut = 1.0

def double_integral_bandpass(time_array, acc_array, dacc_array, lowcut, highcut, fs, order):
    
    butter_lfilter = butter_bandpass_lfilter(dacc_array, lowcut, highcut, fs, order=5)

    
    #First integral is the velocity:
    v_integral = integrate.cumtrapz(x=time_array, y=butter_lfilter, initial=0)
    detrend_v_integral = signal.detrend(v_integral)
    
    v_butter_filter_integral = butter_bandpass_lfilter(detrend_v_integral, lowcut, highcut, fs, order=5)
    detrend_v_integral = signal.detrend(v_butter_filter_integral)


    #Second integral is the displacment:
    disp_integral = integrate.cumtrapz(x=time_array, y=v_butter_filter_integral, initial=0)
    detrend_disp_integral = signal.detrend(disp_integral)
    
    disp_butter_filter_integral = butter_bandpass_lfilter(detrend_disp_integral, lowcut, highcut, fs, order=5)
    detrend_disp_butter_integral = signal.detrend(disp_butter_filter_integral)



    f1 = plt.figure(figsize=(12,5))
    ax1 = f1.add_subplot(121)
    ax2 = f1.add_subplot(122)

    f2 = plt.figure(figsize=(12,5))
    ax3 = f2.add_subplot(121)
    ax4 = f2.add_subplot(122)
    
    f3 = plt.figure(figsize=(12,5))
    ax5 = f3.add_subplot(121)
    ax6 = f3.add_subplot(122)

    #Acceleration graphs:
    ax1.plot(time_array, dacc_array)
    ax1.set_title('Detrended Acceleration vs. Time')
    ax1.set_xlabel('Time [s]')
    ax1.set_ylabel('Velocity [m/s]')
    ax1.axhline(0, color="orange", ls='--')
    
    ax2.plot(time_array, butter_lfilter)
    ax2.set_title('Butter Filtered Detrended Acceleration vs. Time')
    ax2.set_xlabel('Time [s]')
    ax2.set_ylabel('Velocity [m/s]')
    ax2.axhline(0, color="orange", ls='--')
    
    #Velocity graphs:
    ax3.plot(time_array, v_butter_filter_integral)
    ax3.set_title('Butter Filtered Velocity vs. Time')
    ax3.set_xlabel('Time [s]')
    ax3.set_ylabel('Velocity [m/s]')
    ax3.axhline(0, color="orange", ls='--')


    ax4.plot(time_array, detrend_v_integral)
    ax4.set_title('Butter Filtered Detrended Velocity vs. Time')
    ax4.set_xlabel('Time [s]')
    ax4.set_ylabel('Velocity [m/s]')
    ax4.axhline(0, color="orange",ls='--')

    
    #Displacement graphs:
    ax5.plot(time_array, disp_butter_filter_integral)
    ax5.set_title('Butter Filtered Displacement vs. Time')
    ax5.set_xlabel('Time [s]')
    ax5.set_ylabel('Displacement [m]')               
    
    ax6.plot(time_array, detrend_disp_butter_integral)
    ax6.set_title('Detrended Butter Filtered Displacement vs. Time')
    ax6.set_xlabel('Time [s]')
    ax6.set_ylabel('Displacement [m]')


    plt.show()
    
    return detrend_disp_butter_integral

    
    
##For subexperiment1:
print("Starting subexperiment 1 analysis:")
#Get as close to just the signal (don't want noise)    
new_t1, new_acc1 = calculate_new_range(time_array1, acc_array1, 0, 1200)    
new_t1, new_dacc1 = calculate_new_range(time_array1, dacc_array1, 0, 1200)
disp_butter_integral1 = double_integral_bandpass(new_t1, new_acc1, new_dacc1, lowcut, highcut, fs, order)
print("Done.")

##For subexperiment2:
print("Starting subexperiment 2 analysis:")
#Get as close to just the signal (don't want noise)    
new_t2, new_acc2 = calculate_new_range(time_array2, acc_array2, 1200, 2400)    
new_t2, new_dacc2 = calculate_new_range(time_array2, dacc_array2, 1200, 2400)
disp_butter_integral2 = double_integral_bandpass(new_t2, new_acc2, new_dacc2, lowcut, highcut, fs, order)
print("Done.")

## For subexperiment3:
print("Starting subexperiment 3 analysis:")
new_t3, new_acc3 = calculate_new_range(time_array3, acc_array3, 2400, 3600)    
new_t3, new_dacc3 = calculate_new_range(time_array3, dacc_array3, 2400, 3600)
disp_butter_integral3 = double_integral_bandpass(new_t3, new_acc3, new_dacc3, lowcut, highcut, fs, order)


## Using butterworth filter bc it is designed to have a flat frequency response, i.e. most accurately reproducing input through output.

print("Using butterworth filter bc it is designed to have a flat frequency response, i.e. most accurately reproducing input through output.")

## Peak Picking Algorithm:

In [ ]:
#First, find peaks and valleys of the waveforms:
#(Found that this works better when the data has been detrended.)

#Need to convert to numpy array types: 
new_dacc1 = np.array(new_dacc1)
new_dacc2 = np.array(new_dacc2)
new_dacc3 = np.array(new_dacc3)

new_t1 = np.array(new_t1)
new_t2 = np.array(new_t2)
new_t3 = np.array(new_t3)


indexes0 = peakutils.indexes(new_dacc1, thres=0.02/max(new_dacc1), min_dist=100)
indexes1 = peakutils.indexes(new_dacc2, thres=0.02/max(new_dacc2), min_dist=100)
indexes2 = peakutils.indexes(new_dacc3, thres=0.02/max(new_dacc3), min_dist=100)

col_0t = new_t1 # First column data
col_0a = new_dacc1 # Second column data

col_1t = new_t2 # First column data
col_1a = new_dacc2 # Second column data

col_2t = new_t3 # First column data
col_2a = new_dacc3 # Second column data



#Index1 gets the peaks, while index2 gets the valleys
index_max0 = peakutils.indexes(col_0a, thres=0.66, min_dist=25)
index_min0 = peakutils.indexes(-col_0a, thres=0.66, min_dist=25)

index_max1 = peakutils.indexes(col_1a, thres=0.66, min_dist=25)
index_min1 = peakutils.indexes(-col_1a, thres=0.66, min_dist=25)

index_max2 = peakutils.indexes(col_2a, thres=0.66, min_dist=25)
index_min2 = peakutils.indexes(-col_2a, thres=0.66, min_dist=25)



##Plotting:
f1 = plt.figure(figsize=(12,5))
ax1 = f1.add_subplot(121)
ax2 = f1.add_subplot(122)

f2 = plt.figure(figsize=(12,5))
ax3 = f2.add_subplot(121)


ax1.plot(col_0t,col_0a, lw=0.4, alpha=0.2, color="black" )
ax1.plot(col_0t[index_max0],col_0a[index_max0], marker="o", ls="", ms=3, color="red" )
ax1.plot(col_0t[index_min0],col_0a[index_min0], color ="orange", marker="o", ls="", ms=3 )
ax1.set_title("Displacement vs. Time")
ax1.set_xlabel("Time [s]")
ax1.set_ylabel('Displacement [m]')


ax2.plot(col_1t,col_1a, lw=0.4, alpha=0.2, color="black" )
ax2.plot(col_1t[index_max1],col_1a[index_max1], marker="o", ls="", ms=3,  color="red" )
ax2.plot(col_1t[index_min1],col_1a[index_min1], color ="orange", marker="o", ls="", ms=3 )
ax2.set_title("Displacement vs. Time")
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Discplacement [m]")


ax3.plot(col_2t,col_2a, lw=0.4, alpha=0.2, color="black" )
ax3.plot(col_2t[index_max2],col_2a[index_max2], marker="o", ls="", ms=3,  color="red" )
ax3.plot(col_2t[index_min2],col_2a[index_min2], color ="orange", marker="o", ls="", ms=3 )
ax3.set_title("Discplacement vs. Time")
ax3.set_xlabel("Time [s]")
ax3.set_ylabel("Displacement [m]")


plt.show()


### Choose the tallest 1/3 of waves to observe, then take the mean of that subset: 

In [ ]:
print(len(new_dacc1))
print(new_dacc1[:20])
n = int(len(new_dacc1)/3)
print(n)
x0 = new_dacc1
#print (x0[np.argsort(x0)[-n:]])
largest_third = (x0[np.argsort(x0)[-n:]])

print(np.mean(largest_third))
wave_height = np.mean(largest_third)
print("Significant wave height calculated as: ", wave_height)

In [ ]:
print(len(index_max0))
n = int(len(index_max0)/3)
print(n)

max_dacc = []
for i in range(0,len(index_max0)):
    max_dacc.append(new_dacc1[index_max0])

max_dacc = np.array(max_dacc)
largest_third = (max_dacc[np.argsort(max_dacc)[-n:]])


print("Significant wave height calculated as: ", 2*np.mean(largest_third))

# Detrend on entire data

In [ ]:
# acc_array = acc_list = imu2_array = imu_array2

dacc_test_array = signal.detrend(acc_array)

butter_t_lfilter = butter_bandpass_lfilter(dacc_test_array, lowcut, highcut, fs, order=5)
butter_t_filtfilt = butter_bandpass_filtfilt(dacc_test_array, lowcut, highcut, fs, order=5)

##For entire data:
print("Starting full data set analysis:")
#Get as close to just the signal (don't want noise)    
new_t_test, new_acc_test = calculate_new_range(time_array, acc_array, 0, 1800)
new_t_test, new_dacc_test = calculate_new_range(time_array, dacc_test_array, 0, 1800)
disp_butter_integral_t = double_integral_bandpass(new_t_test, new_acc_test, new_dacc_test, lowcut, highcut, fs, order)
print("Done.")

new_dacc = np.array(new_dacc_test)
new_t = np.array(new_t_test)

indices = peakutils.indexes(new_dacc, thres=0.02/max(new_dacc), min_dist=100)

col_t = new_t
col_a = new_dacc

index_max = peakutils.indexes(col_a, thres=0.66, min_dist=25)
index_min = peakutils.indexes(-col_a, thres=0.66, min_dist=25)

f0 = plt.figure(figsize=(12,5))
ax0 = f0.add_subplot(121)

ax0.plot(col_t, col_a, lw=0.4, alpha=0.2, color="black" )
ax0.plot(col_t[index_max], col_a[index_max], marker="o", ls="", ms=3, color="red" )
ax0.plot(col_t[index_min], col_a[index_min], color ="orange", marker="o", ls="", ms=3 )
ax0.set_title("Displacement vs. Time")
ax0.set_xlabel("Time [s]")
ax0.set_ylabel('Displacement [m]')


In [ ]:
n = int(len(new_dacc)/4)
xn = new_dacc
upper_half = (xn[np.argsort(xn)[(2*n):(3*n)]])

wave_height = 2*np.mean(upper_half)
print("Significant wave height calculated as: ", wave_height)

In [ ]:
n = int(len(new_dacc)/3)
xn = new_dacc
largest_third = (xn[np.argsort(xn)[-n:]])

wave_height = np.mean(largest_third)
print("Significant wave height calculated as: ", wave_height)